In [1]:
import torch
from pyqcu.ascend import dslash
from pyqcu.ascend.define import *
# latt_size = (32, 32, 32, 32)
# latt_size = (32, 32, 16, 16)
# latt_size = (16, 16, 16, 32)
# latt_size = (16, 16, 16, 16)
# latt_size = (16, 16, 16, 8)
# latt_size = (16, 16, 8, 8)
# latt_size = (16, 8, 8, 8)
latt_size = (8, 8, 8, 8)
# latt_size = (8, 16, 16, 32)
# latt_size = (32, 32, 32, 32)
# latt_size = (32, 32, 32, 64)
# latt_size = (4, 8, 8, 8)
# latt_size = (8, 8, 8, 4)
# latt_size = (16, 8, 8, 8)
# latt_size = (8, 8, 8, 16)
# latt_size = (8, 8, 8, 8)
# latt_size = (4, 4, 4, 4)
# latt_size = (2, 2, 2, 2)
# mass = -3.5
# mass = -0.8
# mass = -0.5
# mass = 0.05
# mass = 0.0
mass = -0.05
# kappa = 0.4
# kappa = 0.125
# kappa = 0.5
kappa = 1 / (2 * mass + 8)
dtype = torch.complex128
# dtype = torch.complex64
# dtype = torch.complex32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
print(f"Using device: {device}")
# Initialize lattice gauge theory
wilson = dslash.wilson_mg(
    latt_size=latt_size,
    kappa=kappa,
    dtype=dtype,
    device=device,
    verbose=False
)
clover = dslash.clover(
    latt_size=latt_size,
    kappa=kappa,
    dtype=dtype,
    device=device,
    verbose=False
)
U = wilson.generate_gauge_field(sigma=0.1, seed=42)

wilson.check_su3(U)
clover_term = clover.make_clover(U=U)
# clover_term = torch.zeros_like(clover_term) # just for test, just wilson

b = torch.randn(4, 3, latt_size[3], latt_size[2], latt_size[1], latt_size[0],
                dtype=dtype, device=device)
verbose = False
tol = 1e-6

Using device: cpu


In [2]:
hopping_plus_list = []
hopping_minus_list = []
for ward in range(4):
    hopping_plus_list.append(wilson.give_hopping_plus(ward=ward, U=U))
    hopping_minus_list.append(wilson.give_hopping_minus(ward=ward, U=U))
Ab = wilson.give_wilson(src=b, U=U, with_I=False)
_Ab = torch.zeros_like(b)
for ward in range(4):
    _Ab += wilson.give_wilson_plus(ward=ward, src=b.reshape(
        [12]+list(U.shape[-4:])), hopping=hopping_plus_list[ward]).reshape([4, 3]+list(U.shape[-4:]))
    _Ab += wilson.give_wilson_minus(ward=ward, src=b.reshape(
        [12]+list(U.shape[-4:])), hopping=hopping_minus_list[ward]).reshape([4, 3]+list(U.shape[-4:]))
print(torch.norm(Ab-_Ab).item()/torch.norm(Ab).item())

2.163165884495769e-16


In [3]:
from pyqcu.ascend import inverse
mg = inverse.mg(b=b, wilson=wilson, U=U,
                clover=clover, clover_term=clover_term, tol=tol, verbose=verbose)
mg.init()

self.dof_list:[12, 24, 24, 24, 24, 4, 4, 24, 12, 12, 12, 24, 24, 24, 24, 48, 48, 24, 8, 8, 8, 4, 12, 12, 12, 8, 4, 2, 4, 4, 24, 12, 12, 12, 4, 4, 4, 4, 4]


Building grid list:
  Level 0: 8x8x8x8
  Level 1: 4x4x4x4
  Level 2: 2x2x2x2
self.grid_list:[[8, 8, 8, 8], [4, 4, 4, 4], [2, 2, 2, 2]]

Performance Statistics:
Total iterations: 109
Total time: 2.689557 seconds
Average time per iteration: 0.024671 s
Final residual: 4.18e-05

Performance Statistics:
Total iterations: 110
Total time: 2.828383 seconds
Average time per iteration: 0.025709 s
Final residual: 4.95e-05

Performance Statistics:
Total iterations: 98
Total time: 2.485435 seconds
Average time per iteration: 0.025358 s
Final residual: 4.61e-05

Performance Statistics:
Total iterations: 101
Total time: 2.619062 seconds
Average time per iteration: 0.025928 s
Final residual: 4.64e-05

Performance Statistics:
Total iterations: 104
Total time: 2.765226 seconds
Average time per iteration: 0.026585 s
Final residual: 4.55e-05

Performance Statistics:
Total iterations: 112
Total time: 2.684392 seconds
Average time per iteration: 0.023963 s
Final residual: 4.76e-05

Performance Statistics:
T

In [4]:
def matvec(src: torch.Tensor, U: torch.Tensor = U, clover_term: torch.Tensor = clover_term) -> torch.Tensor:
    return wilson.give_wilson(src, U)+clover.give_clover(clover_term=clover_term, src=src)


def _matvec(src: torch.Tensor) -> torch.Tensor:
    return mg.op_list[0].matvec(src=src)

In [5]:
%%time
%time Ab = matvec(b)
%time _Ab = _matvec(b)
print(torch.norm(U).item())
print(torch.norm(clover_term).item())
print(torch.norm(Ab).item())
print(torch.norm(_Ab).item())
print(torch.norm(Ab-_Ab).item()/torch.norm(_Ab).item())

CPU times: user 60.3 ms, sys: 0 ns, total: 60.3 ms
Wall time: 12.1 ms
CPU times: user 83.5 ms, sys: 0 ns, total: 83.5 ms
Wall time: 13.6 ms
221.70250336881637
10.706957214212924
247.6479310188933
247.64793101889333
1.769862410103817e-16
CPU times: user 220 ms, sys: 1 μs, total: 220 ms
Wall time: 31.6 ms


In [6]:
%time Ab = matvec(b)
%time _Ab = _matvec(b)

CPU times: user 82 ms, sys: 0 ns, total: 82 ms
Wall time: 11 ms
CPU times: user 83.8 ms, sys: 0 ns, total: 83.8 ms
Wall time: 13.6 ms


In [7]:
b0 = mg.b_list[0]
b1 = inverse.restrict(
    local_ortho_null_vecs=mg.lonv_list[0], fine_vec=b0)
_b0 = inverse.prolong(local_ortho_null_vecs=mg.lonv_list[0], coarse_vec=b1)
_b1 = inverse.restrict(
    local_ortho_null_vecs=mg.lonv_list[0], fine_vec=_b0)
print(_b1.flatten()[:100]/b1.flatten()[:100])

restrict:shape,coarse_dof:(torch.Size([24, 12, 4, 2, 4, 2, 4, 2, 4, 2]), 24)
EeTtZzYyXx,eTtZzYyXx->ETZYX
prolong:shape,fine_dof:(torch.Size([24, 12, 4, 2, 4, 2, 4, 2, 4, 2]), 12)
EeTtZzYyXx,ETZYX->eTtZzYyXx
restrict:shape,coarse_dof:(torch.Size([24, 12, 4, 2, 4, 2, 4, 2, 4, 2]), 24)
EeTtZzYyXx,eTtZzYyXx->ETZYX
tensor([1.0000-4.1366e-17j, 1.0000-1.3759e-16j, 1.0000+1.1061e-16j,
        1.0000+1.3543e-16j, 1.0000+4.3582e-16j, 1.0000-3.5252e-16j,
        1.0000+0.0000e+00j, 1.0000+1.2319e-16j, 1.0000-2.7530e-17j,
        1.0000-2.3271e-16j, 1.0000+1.6351e-16j, 1.0000-1.8225e-16j,
        1.0000+4.7166e-16j, 1.0000+2.6315e-16j, 1.0000-1.3335e-16j,
        1.0000+1.4609e-16j, 1.0000+2.3377e-16j, 1.0000-8.6462e-17j,
        1.0000+4.8876e-16j, 1.0000+1.5021e-16j, 1.0000+0.0000e+00j,
        1.0000+5.0103e-16j, 1.0000+4.1617e-16j, 1.0000-2.0753e-16j,
        1.0000-1.4325e-16j, 1.0000+2.5953e-16j, 1.0000-1.0557e-16j,
        1.0000+5.4345e-16j, 1.0000+1.9128e-16j, 1.0000+3.0259e-16j,
        

In [8]:
# _x = inverse.cg(b=b, matvec=matvec, tol=tol, verbose=verbose)
_x = inverse.bicgstab(b=b, matvec=_matvec, tol=tol, verbose=verbose)
# _x = inverse.bicgstab(b=b, matvec=mg.op_list[0].matvec, tol=tol, verbose=verbose)


Performance Statistics:
Total iterations: 191
Total time: 4.675019 seconds
Average time per iteration: 0.024473 s
Final residual: 8.39e-07


In [20]:
mg.num_levels = 3
x = mg.solve()
mg.plot()


Performance Statistics:
Total iterations: 11
Total time: 0.019264 seconds
Average time per iteration: 0.001750 s
Final residual: 6.88e-01

Performance Statistics:
Total iterations: 13
Total time: 0.024262 seconds
Average time per iteration: 0.001864 s
Final residual: 1.57e-01

Performance Statistics:
Total iterations: 2
Total time: 0.152694 seconds
Average time per iteration: 0.076340 s
Final residual: 4.17e+00

Performance Statistics:
Total iterations: 7
Total time: 0.013002 seconds
Average time per iteration: 0.001855 s
Final residual: 1.37e-01

Performance Statistics:
Total iterations: 14
Total time: 0.021626 seconds
Average time per iteration: 0.001543 s
Final residual: 2.06e-02

Performance Statistics:
Total iterations: 2
Total time: 0.136769 seconds
Average time per iteration: 0.068379 s
Final residual: 1.15e+00

Performance Statistics:
Total iterations: 9
Total time: 0.014183 seconds
Average time per iteration: 0.001574 s
Final residual: 6.30e-02

Performance Statistics:
Total 

In [10]:
mg.lonv_list[0].shape

torch.Size([24, 12, 4, 2, 4, 2, 4, 2, 4, 2])

In [11]:
(mg.b_list[0]-mg.op_list[0].matvec(x.reshape([12]+list(x.shape[-4:])))).flatten()[:100]

tensor([-6.6917e-10+2.3874e-09j,  1.8980e-09+7.5353e-10j,
        -3.7028e-10+1.9098e-10j, -9.0242e-10-1.0069e-09j,
        -2.0661e-09-6.0329e-10j, -1.5689e-09+3.3531e-10j,
        -2.7637e-09+1.4204e-09j,  1.0988e-09+1.3266e-09j,
         1.5776e-09-4.8875e-10j, -4.4834e-10+1.6601e-09j,
         1.4987e-09-1.5450e-09j,  1.0550e-09+2.5190e-10j,
        -1.1633e-10+8.1513e-10j, -1.4395e-09-4.1044e-10j,
        -1.1309e-09-6.2547e-10j,  7.8870e-11-1.0249e-09j,
        -4.8620e-10+9.8130e-10j,  2.3637e-09-3.0849e-10j,
         7.4242e-10+2.8848e-10j,  7.8725e-10+3.0565e-10j,
         3.8831e-10+1.5114e-09j, -8.3557e-10-1.1517e-09j,
        -1.1346e-09+1.8545e-09j,  2.2343e-09-3.5623e-11j,
         1.5632e-09-1.0810e-09j, -1.3251e-10+8.2783e-10j,
         4.1693e-10+1.1048e-09j, -1.7007e-09+1.6170e-09j,
        -2.2564e-09+1.5286e-09j, -1.8836e-09+1.6700e-09j,
        -1.5878e-09-1.3260e-09j, -8.6569e-10+1.1984e-09j,
        -1.5815e-09-7.0530e-10j,  1.3274e-09-1.0795e-09j,
        -1.464

In [12]:
print(torch.norm(x-_x).item()/torch.norm(_x).item())

1.4499735392477283e-07


In [13]:
mg.num_levels

2

In [14]:
index = 0
null_vecs = mg.nv_list[index]
matvec = mg.op_list[index].matvec
for i in range(null_vecs.shape[0]):
    print(
        f"(matvec(null_vecs[i])/null_vecs[i]).flatten()[:10]:{(matvec(null_vecs[i])/null_vecs[i]).flatten()[:10]}")

(matvec(null_vecs[i])/null_vecs[i]).flatten()[:10]:tensor([0.0154+0.0026j, 0.0138+0.0028j, 0.0142+0.0018j, 0.0107+0.0043j,
        0.0148+0.0013j, 0.0107+0.0051j, 0.0149+0.0006j, 0.0160+0.0020j,
        0.0121-0.0008j, 0.0131-0.0001j], dtype=torch.complex128)
(matvec(null_vecs[i])/null_vecs[i]).flatten()[:10]:tensor([0.0075-0.0033j, 0.0077-0.0027j, 0.0087-0.0026j, 0.0089-0.0015j,
        0.0097-0.0008j, 0.0086-0.0013j, 0.0082-0.0017j, 0.0076-0.0030j,
        0.0082-0.0026j, 0.0088-0.0027j], dtype=torch.complex128)
(matvec(null_vecs[i])/null_vecs[i]).flatten()[:10]:tensor([0.0127-0.0104j, 0.0082-0.0094j, 0.0093-0.0094j, 0.0084-0.0071j,
        0.0069-0.0043j, 0.0063-0.0004j, 0.0057-0.0013j, 0.0089-0.0040j,
        0.0125-0.0057j, 0.0117-0.0091j], dtype=torch.complex128)
(matvec(null_vecs[i])/null_vecs[i]).flatten()[:10]:tensor([0.0079+1.6436e-03j, 0.0074+2.2021e-04j, 0.0085-1.7757e-03j,
        0.0072+1.4182e-05j, 0.0080-2.3940e-03j, 0.0050-2.0145e-03j,
        0.0064+2.1950e-03j, 0.006

In [15]:
mg.nv_list[-1].flatten()[:100]

tensor([-0.0203-0.0162j, -0.0219-0.0186j, -0.0198-0.0170j, -0.0183-0.0188j,
        -0.0197-0.0183j, -0.0207-0.0208j, -0.0193-0.0186j, -0.0186-0.0172j,
        -0.0205-0.0194j, -0.0198-0.0187j, -0.0177-0.0193j, -0.0184-0.0175j,
        -0.0231-0.0180j, -0.0204-0.0183j, -0.0168-0.0188j, -0.0201-0.0165j,
        -0.0208-0.0190j, -0.0187-0.0168j, -0.0184-0.0173j, -0.0211-0.0194j,
        -0.0217-0.0202j, -0.0189-0.0171j, -0.0191-0.0166j, -0.0206-0.0178j,
        -0.0219-0.0184j, -0.0189-0.0175j, -0.0176-0.0192j, -0.0191-0.0179j,
        -0.0193-0.0219j, -0.0185-0.0179j, -0.0182-0.0180j, -0.0190-0.0181j,
        -0.0214-0.0191j, -0.0181-0.0175j, -0.0185-0.0174j, -0.0194-0.0188j,
        -0.0209-0.0168j, -0.0189-0.0170j, -0.0191-0.0184j, -0.0198-0.0188j,
        -0.0203-0.0186j, -0.0191-0.0185j, -0.0177-0.0185j, -0.0172-0.0181j,
        -0.0199-0.0187j, -0.0198-0.0178j, -0.0176-0.0178j, -0.0173-0.0192j,
        -0.0192-0.0175j, -0.0181-0.0189j, -0.0180-0.0169j, -0.0156-0.0182j,
        -0.0

In [16]:
_null_vec = torch.randn_like(null_vecs[0])
print(torch.norm(matvec(_null_vec)))
print(matvec(_null_vec).flatten()[:20])
_null_vec -= inverse.bicgstab(b=matvec(_null_vec), matvec=matvec, tol=5e-5)
print(torch.norm(_null_vec))
print(_null_vec.flatten()[:20])
_null_vec/=torch.norm(_null_vec).item()
print(
    f"(matvec(_null_vec)/_null_vec).flatten()[:10]:{(matvec(_null_vec)/_null_vec).flatten()[:10]}")

tensor(248.2552, dtype=torch.float64)
tensor([-3.8271e-01-0.4378j,  3.0290e-04-0.1331j,  6.5963e-01-0.3197j,
         9.4558e-01-0.2566j, -3.0660e-01+0.8403j,  5.2489e-01+1.2023j,
         5.8727e-03-1.0743j,  1.0890e+00+0.0192j, -1.2506e+00+1.0300j,
         2.3735e-01+0.5066j, -3.5632e-02+0.3881j, -9.3677e-01+0.1106j,
         2.6242e-01-0.9151j, -6.7273e-02-1.1906j,  7.8959e-01+0.6298j,
         4.9399e-01-0.7980j,  1.8558e+00-1.7031j,  8.0127e-02-0.9544j,
        -4.6462e-01-0.3518j, -4.1259e-01+1.4254j], dtype=torch.complex128)
Norm of b:248.25521098019433
Norm of r:352.5024549067647
Norm of x0:221.00962312239608
BICGSTAB-Iteration 0: Residual = 6.075991e+01, Time = 0.026301 s
BICGSTAB-Iteration 1: Residual = 2.025648e+01, Time = 0.023030 s
BICGSTAB-Iteration 2: Residual = 4.850067e+01, Time = 0.024972 s
BICGSTAB-Iteration 3: Residual = 6.634396e+00, Time = 0.024160 s
BICGSTAB-Iteration 4: Residual = 5.028532e+00, Time = 0.024022 s
BICGSTAB-Iteration 5: Residual = 2.877711e+00, Ti

In [17]:
_b = torch.randn_like(null_vecs[0])
_x = inverse.bicgstab(b=_b, matvec=matvec, tol=1e-8)
print((_b-matvec(_x)).flatten()[:100])
print(torch.norm(_b))
print(torch.norm(_x))

Norm of b:221.98868290586267
Norm of r:333.3720321094754
Norm of x0:222.02982162341434


BICGSTAB-Iteration 0: Residual = 7.219609e+01, Time = 0.030331 s
BICGSTAB-Iteration 1: Residual = 3.499176e+01, Time = 0.028681 s
BICGSTAB-Iteration 2: Residual = 3.061004e+01, Time = 0.027584 s
BICGSTAB-Iteration 3: Residual = 2.070745e+01, Time = 0.027315 s
BICGSTAB-Iteration 4: Residual = 1.306081e+01, Time = 0.029094 s
BICGSTAB-Iteration 5: Residual = 9.732874e+00, Time = 0.024363 s
BICGSTAB-Iteration 6: Residual = 9.318352e+00, Time = 0.028551 s
BICGSTAB-Iteration 7: Residual = 8.591149e+00, Time = 0.028453 s
BICGSTAB-Iteration 8: Residual = 9.230711e+00, Time = 0.026923 s
BICGSTAB-Iteration 9: Residual = 4.921352e+00, Time = 0.025558 s
BICGSTAB-Iteration 10: Residual = 1.568130e+01, Time = 0.026404 s
BICGSTAB-Iteration 11: Residual = 1.633497e+01, Time = 0.032446 s
BICGSTAB-Iteration 12: Residual = 3.824455e+01, Time = 0.028497 s
BICGSTAB-Iteration 13: Residual = 3.763681e+01, Time = 0.025323 s
BICGSTAB-Iteration 14: Residual = 2.816958e+01, Time = 0.026353 s
BICGSTAB-Iteration 1

In [18]:
_null_vec.flatten()[:100]

tensor([0.0019-1.0256e-03j, 0.0021+9.4540e-05j, 0.0024-3.3816e-04j,
        0.0028-3.4458e-04j, 0.0026-4.7556e-04j, 0.0017-3.9379e-04j,
        0.0021-2.5878e-04j, 0.0024-9.4306e-04j, 0.0020-7.5944e-04j,
        0.0023-3.0640e-04j, 0.0024-9.7389e-04j, 0.0020-1.3136e-03j,
        0.0023-8.6710e-04j, 0.0021-3.0868e-04j, 0.0026-9.3663e-04j,
        0.0018-5.8757e-04j, 0.0019-6.7431e-04j, 0.0021-9.8634e-04j,
        0.0019-1.2858e-03j, 0.0020-1.3195e-03j, 0.0022-9.7868e-04j,
        0.0019-5.3213e-04j, 0.0024-9.9275e-04j, 0.0020-1.1991e-03j,
        0.0019-2.8694e-04j, 0.0018-4.4055e-04j, 0.0020-9.7843e-04j,
        0.0023-6.6597e-04j, 0.0019-2.9637e-04j, 0.0026-3.9009e-04j,
        0.0029-6.3376e-04j, 0.0025-1.3437e-03j, 0.0024-4.9012e-04j,
        0.0022-1.1514e-03j, 0.0022-1.3180e-03j, 0.0026-5.3518e-04j,
        0.0022-5.6881e-04j, 0.0023-7.1212e-04j, 0.0025-9.6336e-04j,
        0.0023-1.3604e-03j, 0.0019-3.9339e-04j, 0.0021-5.7357e-04j,
        0.0028-9.9418e-04j, 0.0027-1.2298e-03j, 